In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def evaluate_rules(data, tuned_rules, output):
    predictions_all = []
    for i, rule in tuned_rules.iterrows():
        pred_rule = []
        tptot = 0
        tntot = 0
        fntot = 0
        fptot = 0
        tuned_antecedent = rule['Rule'].strip()
        print(f"rule: {tuned_antecedent}")
        for _, row in data.iterrows():
            y_row = row[output]
            tp = 0
            tn = 0
            fn = 0
            fp = 0
            # check if the point row satifies rule 
            if evaluate_rule_conditions(row, tuned_antecedent):
                # rule is satisfied
                satisfied = True
                pred_rule.append(0)
            else:
                satisfied = False
                pred_rule.append(1)
    
            
            if satisfied and y_row == 0:
                tp=1
            elif satisfied and y_row == 1:
                fp=1
            elif not satisfied and y_row == 1:
                tn=1
            
            elif not satisfied and y_row == 0:
                fn=1

            predictions_all.append(pred_rule)
            tptot+=tp
            tntot+=tn
            fptot+=fp
            fntot+=fn

        if (tptot+fptot) == 0:
            print("no points satisfy the rule\n")
            continue
        else:
            
            print(f"tp = {tptot}, tn = {tntot}, fp = {fptot}, fn = {fntot}")
            prec = tptot / (tptot+fptot)
            cov = tptot/(tptot+fntot)
            acc = (tptot+tntot)/(tptot+tntot+fptot+fntot)
            f1score = (2*tptot)/(2*tptot+fptot+fntot)
            error = fptot/(fptot+tntot)
            print(f"precision: {prec}, covering: {cov}, error: {error}, accuracy: {acc}, f1: {f1score}\n")              

    return predictions_all


def evaluate_rule_conditions(row, condition_part):

    # Checks if any of the conditions in the rule are satisfied
    if all(check_condition(row, part) for part in condition_part.split(" and ")):
        return True  # Return True if all conditions in the rule are satisfied
    
    return False  # Return False if any of the conditions in the rule is not satisfied


def check_condition(row, condition_part):
    # Check if a single condition part is satisfied
    
    #parts = [part.strip('()') for part in condition_part.split()]
    parts = re.split(r'\s*(==|<=|>=|<|>|!=)\s*', condition_part.strip('()'))
    #print("parts: ", parts)
    if len(parts) == 3:
        column, op, value = parts
        return eval(f"{row[column]} {op} {value}")
    # handle the case of a 2-thresholds conditions of the kind: a < Column <= b
    elif len(parts) == 5:
        val1,op1,column,op2,val2 = parts
        # Use the original condition from the rule
        return eval(f"{val1} {op1} {row[column]} {op2} {val2}")        
    else:
        # Handle cases where the condition is not in the expected format
        pass


In [3]:
# Loading rules actual rules
rules = pd.read_csv('/Users/saranarteni/OneDrive - CNR/REXASI-PRO/T3.1/navground_code/simulation2/confiderai/supsinavground_rules.csv', header=None, names=["Rule", "Covering", "Error"])

# adjust columns values
for i in range(len(rules)+1):   
    rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("RULE {}: ".format(i),""))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("AND","and"))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("{",""))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("}",""))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("output in ","output = "))
rules["Covering"] = rules["Covering"].apply(lambda x: x.replace("COVERING: ",""))
rules["Error"] = rules["Error"].apply(lambda x: x.replace("ERROR: ",""))
rules['Output'] = rules['Rule'].str.extract(r'output = (\d+)', expand=False).astype(int)
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("IF ",""))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace(x[x.find("THEN"):],""))
# load test data
data = pd.read_csv("simulation2/test.csv")

rules_noncoll = rules[rules["Output"]==0]
preds_LLM = evaluate_rules(data, rules_noncoll, "output")

rule: SafetyMargin > 0.071752 and Tau <= 0.798864
tp = 219, tn = 431, fp = 18, fn = 332
precision: 0.9240506329113924, covering: 0.397459165154265, error: 0.0400890868596882, accuracy: 0.65, f1: 0.5558375634517766

rule: SafetyMargin > 0.019271 and 0.096355 < Tau <= 0.359004
tp = 216, tn = 429, fp = 20, fn = 335
precision: 0.9152542372881356, covering: 0.39201451905626133, error: 0.044543429844097995, accuracy: 0.645, f1: 0.5489199491740788

rule: SafetyMargin > 0.070627 and Eta > 0.372908
tp = 192, tn = 441, fp = 8, fn = 359
precision: 0.96, covering: 0.3484573502722323, error: 0.017817371937639197, accuracy: 0.633, f1: 0.511318242343542

rule: 0.041272 < SafetyMargin <= 0.071004 and Tau <= 0.619041
tp = 142, tn = 434, fp = 15, fn = 409
precision: 0.9044585987261147, covering: 0.2577132486388385, error: 0.0334075723830735, accuracy: 0.576, f1: 0.4011299435028249

rule: SafetyMargin > 0.010931 and Eta > 0.748042 and Tau <= 0.682408
tp = 145, tn = 430, fp = 19, fn = 406
precision: 0.884

# Skope Rules performance

In [3]:
#mix both rules, model for collision and model for no collision.
skrules_unsafe = pd.read_csv('simulationVictorTopological/skope/skope_rules_unsafety.csv', header=None, names=["Rule","Covering","Error"])
skrules_safe = pd.read_csv('simulationVictorTopological/skope/skope_rules_safety.csv', names=["Rule","Covering","Error"])

skrules = pd.concat([skrules_unsafe, skrules_safe], ignore_index=True)
skrules

,Rule,Covering,Error
0,RULE 1: IF meanEntropy > 2.767314910888672 and...,COVERING: 0.5380997177798683,ERROR: 0.0034843205574912606
1,RULE 2: IF meanEntropy > 2.7700753211975098 TH...,COVERING: 0.5096889913593425,ERROR: 0.0006904406738674762
2,RULE 3: IF meanEntropy <= 2.671537399291992 an...,COVERING: 0.10056390977443609,ERROR: 0.1336032388663968
3,RULE 4: IF meanEntropy <= 2.6690224409103394 a...,COVERING: 0.09416195856873823,ERROR: 0.1228070175438597
4,RULE 5: IF meanEntropy <= 2.7657963037490845 a...,COVERING: 0.09411764705882353,ERROR: 0.11894273127753308
5,RULE 6: IF meanEntropy <= 2.6690224409103394 a...,COVERING: 0.09290382819794585,ERROR: 0.1076233183856502
6,RULE 7: IF medianEntropy <= 2.7564250230789185...,COVERING: 0.08795860771401694,ERROR: 0.15000000000000002
7,RULE 8: IF meanEntropy <= 2.7660937309265137 a...,COVERING: 0.08578088578088579,ERROR: 0.0980392156862745
8,RULE 9: IF meanEntropy <= 2.766075015068054 an...,COVERING: 0.0847378277153558,ERROR: 0.0670103092783505
9,RULE 10: IF medianEntropy <= 2.65862500667572 ...,COVERING: 0.07149532710280374,ERROR: 0.0892857142857143


In [5]:
skrules["Rule"][0]

'RULE 1: IF meanEntropy > 2.767314910888672 and medianEntropy > 2.757949948310852 THEN output = 1'

In [6]:
# adjust columns values
for i in range(len(skrules)+1):   
    skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("RULE {}: ".format(i),""))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("AND","and"))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("{",""))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("}",""))
#skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("output in ","output = "))
skrules["Covering"] = skrules["Covering"].apply(lambda x: x.replace("COVERING: ",""))
skrules["Error"] = skrules["Error"].apply(lambda x: x.replace("ERROR: ",""))
skrules['Output'] = skrules['Rule'].str.extract(r'output = (\d+)', expand=False).astype(int)
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("IF ",""))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace(x[x.find("THEN"):],""))

In [8]:
# for safe simulations 
data = pd.read_csv("simulationVictorTopological/test.csv")
skrules_safe = skrules[skrules["Output"]==0]
preds_skope = evaluate_rules(data, skrules_safe, "output")

rule: meanEntropy > 2.6634804010391235 and medianEntropy <= 2.757949948310852 and iqrsEntropy <= 0.040037499740719795
tp = 271, tn = 568, fp = 108, fn = 53
precision: 0.7150395778364116, covering: 0.8364197530864198, error: 0.15976331360946747, accuracy: 0.839, f1: 0.7709815078236131

rule: medianEntropy <= 2.756624937057495 and medianEntropy > 2.6713500022888184 and stdsEntropy <= 0.02991117537021637
tp = 270, tn = 578, fp = 98, fn = 54
precision: 0.7336956521739131, covering: 0.8333333333333334, error: 0.14497041420118342, accuracy: 0.848, f1: 0.7803468208092486

rule: meanEntropy <= 2.754297971725464 and meanEntropy > 2.6722307205200195 and stdsEntropy <= 0.029967069625854492
tp = 268, tn = 577, fp = 99, fn = 56
precision: 0.7302452316076294, covering: 0.8271604938271605, error: 0.14644970414201183, accuracy: 0.845, f1: 0.7756874095513748

rule: meanEntropy <= 2.753678560256958 and medianEntropy > 2.668500065803528 and stdsEntropy <= 0.028100933879613876
tp = 242, tn = 592, fp = 84,

In [9]:
# for unsafe simulations
data = pd.read_csv("simulationVictorTopological/test.csv")
skrules_unsafe = skrules[skrules["Output"]==1]
preds_skope = evaluate_rules(data, skrules_unsafe, "output")

rule: meanEntropy > 2.767314910888672 and medianEntropy > 2.757949948310852
tp = 5, tn = 296, fp = 380, fn = 319
precision: 0.012987012987012988, covering: 0.015432098765432098, error: 0.5621301775147929, accuracy: 0.301, f1: 0.014104372355430184

rule: meanEntropy > 2.7700753211975098
tp = 2, tn = 317, fp = 359, fn = 322
precision: 0.00554016620498615, covering: 0.006172839506172839, error: 0.5310650887573964, accuracy: 0.319, f1: 0.00583941605839416

rule: meanEntropy <= 2.671537399291992 and medianEntropy <= 2.757949948310852 and stdsEntropy > 0.028046048246324062
tp = 8, tn = 612, fp = 64, fn = 316
precision: 0.1111111111111111, covering: 0.024691358024691357, error: 0.09467455621301775, accuracy: 0.62, f1: 0.04040404040404041

rule: meanEntropy <= 2.6690224409103394 and medianEntropy <= 2.7578500509262085 and iqrsEntropy > 0.03773749992251396
tp = 8, tn = 614, fp = 62, fn = 316
precision: 0.11428571428571428, covering: 0.024691358024691357, error: 0.09171597633136094, accuracy: 0.